In [1]:
import math
from copy import deepcopy
import numpy as np
import types
import pylab
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly.graph_objs import Surface
import plotly.tools as tls
import sympy
from sympy import *
tls.set_credentials_file(username='Mungos', api_key='mcnflyescl')

In [2]:
class Function():
    def subs_dict(self,x):
        dic = {}
        for i,var in enumerate(self.variables):
            dic[var] = x[i]
        return dic
    
    def gradient_symbolic(self):
        grads = []
        for var in self.variables:
            dif = diff(self.equation,var)
            grads.append(dif)
        return grads
        
    def gradient(self,x):
        self.gradient_calls+=1
        sub_dict = self.subs_dict(x)
        grads = self.gradient_symbolic()
        for i,grad in enumerate(grads):
            grads[i]=grad.evalf(subs=sub_dict)
        return np.array(grads,dtype=np.float32)
        
    def hessean_symbolic(self):
        grads = self.gradient_symbolic()
        hessean = []
        for i,grad in enumerate(grads):
            hess_row = []
            for var in self.variables:
                dif = diff(grad,var)
                hess_row.append(dif)
            hessean.append(hess_row)
        return hessean
        
    def hessean(self,x):
        self.hessean_calls+=1
        hess = self.hessean_symbolic()
        sub_dict = self.subs_dict(x)
        for i,row in enumerate(hess):
            for j, val in enumerate(row):
                hess[i][j] = hess[i][j].evalf(subs=sub_dict)
        return np.array(hess,dtype=np.float32)
        
    def evaluate(self,x, exponential=False):
        self.calls+=1
        subs = self.subs_dict(x)
        value = np.array(self.equation.evalf(subs=subs),dtype=np.float32)
        return value

In [3]:
class Rosenbrock(Function):
    def __init__(self):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        self.equation = (100*(self.x2-self.x1**2)**2+(1-self.x1)**2)

In [4]:
class F3(Function):
    def __init__(self, num_vars):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.variables = []
        self.equation = 0
        for i, var in enumerate(range(num_vars)):
            variable = symbols('x'+str(var+1))
            self.variables.append(variable)
            self.equation += (variable-i-1)**2

In [5]:
class F6(Function):
    def __init__(self, num_vars):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.variables = []
        self.sum_vars = 0
        for i, var in enumerate(range(num_vars)):
            variable = symbols('x'+str(var+1))
            self.variables.append(variable)
            self.sum_vars += (variable)**2
            
        self.equation = 0.5 + ((sin(sqrt(self.sum_vars)))**2-0.5)/(1+0.001*self.sum_vars)

In [6]:
class F7(Function):
    def __init__(self, num_vars):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.variables = []
        self.sum_vars = 0
        for i, var in enumerate(range(num_vars)):
            variable = symbols('x'+str(var+1))
            self.variables.append(variable)
            self.sum_vars += (variable)**2
            
        self.equation = self.sum_vars**0.25 * (1+(sin(50*self.sum_vars**0.1))**2)

In [32]:
class OperatoriFloat():
    def __init__(self, first_cross=True, mut_chance=0.05):
        self.first_cross = True
        self.mutation_chance = mut_chance
        
    def cross(self, first_parent, second_parent):
        if self.first_cross:
            return self.crossover_1(first_parent, second_parent)
        return self.crossover_2(first_parent, second_parent)
    
    def crossover_1(self, first_parent, second_parent):
        cross_index = np.random.randint(0,first_parent.value.shape[0])
        cromosome_length = first_parent.value.shape[0]
        first_parent_copy = first_parent.duplicate()
        second_parent_copy = second_parent.duplicate()
        for i in range(cromosome_length):
            first_parent_copy.value[i] = (first_parent.value[i]+second_parent.value[i])/2.
        first_parent_copy.check_boundaries()
        return [first_parent_copy]
        
    def crossover_2(self, first_parent, second_parent):
        pass
    
    def mutate(self, child):
        cromosome_length = child.value.shape[0]
        for i in range(cromosome_length):
            if np.random.ranf() < self.mutation_chance:
                child.value[i] = np.random.uniform(child.lower_bound,child.upper_bound)
        return child

class OperatoriBinary():
    def __init__(self, first_cross=True,mut_chance=0.05):
        self.first_cross = True
        self.mutation_chance = mut_chance
        
    def cross(self, first_parent, second_parent):
        if self.first_cross:
            return self.crossover_1(first_parent, second_parent)
        return self.crossover_2(first_parent, second_parent)
    
    def crossover_1(self, first_parent, second_parent):
        cross_index = np.random.randint(0,first_parent.value.shape[0])
        cromosome_length = first_parent.value.shape[0]
        first_parent_copy = first_parent.duplicate()
        second_parent_copy = second_parent.duplicate()
        for i in range(cross_index,cromosome_length):
            first_parent_copy.value[i] = second_parent.value[i]
            second_parent_copy.value[i] = first_parent.value[i]
        return [first_parent_copy,second_parent_copy]
        
        
    def crossover_2(self, first_parent, second_parent):
        pass
    
    def mutate(self, child):
        cromosome_length = child.value.shape[0]
        for i in range(cromosome_length):
            if np.random.ranf() < self.mutation_chance:
                child.value[i] = 1-child.value[i]
        return child

In [28]:
class BinaryUnit():
    def __init__(self, upper_bound, lower_bound, num_vars, precision, value=np.array([])):
        self.fitness = 0
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound
        self.num_vars = num_vars
        self.precision = precision
        if value.shape[0]==0:
            self.value = self.init_random()
        else:
            self.value = value
        
    def check_boundaries(self):
        pass
        
    def init_random(self):
        array = []
        for var in range(self.num_vars):
            for prec in range(self.precision):
                array.append(np.random.choice([0,1]))
        return np.array(array)
        
    def evaluate(self, function):
        self.fitness = function.evaluate(self.decode())
        
    def decode(self):
        array = []
        for var in range(self.num_vars):
            val = 0
            start_ind = var*self.precision
            for prec in range(self.precision):
                if self.value[start_ind + prec] == 1:
                    val+=math.pow(2,self.precision-prec-1)
            val = self.lower_bound + (val * 1./(math.pow(2,self.precision)-1))*(self.upper_bound-self.lower_bound)
            array.append(val)
        
        return np.array(array)
    
    def duplicate(self):
        value = self.value.copy()
        return BinaryUnit(self.upper_bound,self.lower_bound,self.num_vars,self.precision,value)
    
    def encode(self):
        pass

class FloatingPointUnit():
    def __init__(self, upper_bound, lower_bound, num_vars, value=np.array([])):
        self.fitness = 0
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound
        self.num_vars = num_vars
        if value.shape[0]==0:
            self.value = self.init_random()
        else:
            self.value = value
        
    def init_random(self):
        array = []
        for var in range(self.num_vars):
            array.append(self.lower_bound+(self.upper_bound-self.lower_bound)*np.random.ranf())
        return np.array(array)
    
    def check_boundaries(self):
        for i,value in enumerate(self.value):
            if value > self.upper_bound:
                self.value[i] = self.upper_bound
            if value < self.lower_bound:
                self.value[i] = self.lower_bound
        
    def evaluate(self, function):
        self.fitness = function.evaluate(self.decode())
    
    def decode(self):
        return self.value
    
    
    def duplicate(self):
        value = self.value.copy()
        return FloatingPointUnit(self.upper_bound,self.lower_bound,self.num_vars,value)

In [9]:
class TournamentSelection():
    def __init__(self, num_candidates):
        self.num_candidates = num_candidates
        
    def choose_parent(self, population):
        pop_size = len(population)
        best_found_index = np.random.randint(0,pop_size)
        for i in range(self.num_candidates-1):
            index = np.random.randint(0,pop_size)
            if population[index].fitness < population[best_found_index].fitness:
                best_found_index = index
        return population[best_found_index]

In [10]:
class GeneticAlg():
    
    def __init__(self, pop_size, function, upper_bound, lower_bound, num_vars, precision, number_candidates_tournament, binary_rep = True, first_cross=True, mutation_chance = 0.05, max_evals = 10000, verbose=False):
        self.function = function
        self.pop_size = pop_size
        self.num_vars = num_vars
        self.precision = int(math.ceil(math.log(1+(upper_bound-lower_bound)*10**precision)/math.log(2)))
        self.mutation_chance = mutation_chance
        self.max_evals = max_evals
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound
        self.tournament = TournamentSelection(number_candidates_tournament)
        self.verbose = verbose
        if binary_rep:
            self.operators = OperatoriBinary(first_cross,mutation_chance)
            self.population = self.init_random_pop(binary_rep)
        else:
            self.operators = OperatoriFloat(first_cross,mutation_chance)
            self.population = self.init_random_pop(binary_rep)
            
        
    def run(self):
        best = None
        key = lambda x: x.fitness
        best_fitness = float('inf')
        best = None
        while self.function.calls < self.max_evals:
            self.population.sort(key = key)
            if best_fitness > self.population[0].fitness:
                best_fitness = self.population[0].fitness
                best = self.population[0]
                if self.verbose:
                    print "######\nBroj evaluacija: %d\nFitness najbolje jedinke: %s\nNajbolja jedinka: %s\nNajbolja jedinka dekodirana: %s\n" % (self.function.calls, best_fitness, str(self.population[0].value), self.population[0].decode())
            self.new_population(elitism=True)
            
        print "######\nBroj evaluacija: %d\nFitness najbolje jedinke: %s\nNajbolja jedinka: %s\nNajbolja jedinka dekodirana: %s\n" % (self.function.calls, best_fitness, str(self.population[0].value), self.population[0].decode())
        return best
    
    def init_random_pop(self, binary_rep = True):
        pop = []
        while len(pop) < self.pop_size:
            if binary_rep:
                unit = BinaryUnit(self.upper_bound,self.lower_bound,self.num_vars,self.precision)
                unit.evaluate(self.function)
                pop.append(unit)
            else:
                unit = FloatingPointUnit(self.upper_bound,self.lower_bound,self.num_vars)
                unit.evaluate(self.function)
                pop.append(unit)
        return pop
    
    def new_population(self, elitism=False):
        new_pop = []
        if elitism:
            new_pop.append(self.population[0])
            
        while len(new_pop) < self.pop_size:
            par1 = self.tournament.choose_parent(self.population)
            par2 = self.tournament.choose_parent(self.population)
            children = self.operators.cross(par1,par2)
            for child in children:
                child = self.operators.mutate(child)
                child.evaluate(self.function)
                new_pop.append(child)
            
        self.population = new_pop

# 1. zadatak

### Rosenbrock

In [48]:
lower = -50
upper = 150
prec = 4
num_vars = 2
num_cand = 3
pop = 500
mut = 0.1
fun = Rosenbrock()
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=100000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 100300
Fitness najbolje jedinke: 0.000199493937544
Najbolja jedinka: [0 1 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 1
 1 0 1 1 1]
Najbolja jedinka dekodirana: [ 1.00863028  1.01845313]



<__main__.BinaryUnit instance at 0x10ff8e488>

### F3

In [51]:
lower = -50
upper = 150
prec = 4
num_vars = 5
num_cand = 3
pop = 500
mut = 0.05
fun = F3(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=100000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 100300
Fitness najbolje jedinke: 0.158713847399
Najbolja jedinka: [0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1 1
 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 1 1 1 1 0 1 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0]
Najbolja jedinka dekodirana: [ 1.07824854  1.85434907  3.28390755  4.16557987  4.84717123]



<__main__.BinaryUnit instance at 0x1109dc6c8>

### F6

In [61]:
lower = -50
upper = 150
prec = 4
num_vars = 2
num_cand = 3
pop = 500
mut = 0.05
fun = F6(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=100000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 100000
Fitness najbolje jedinke: 2.59904322775e-06
Najbolja jedinka: [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0]
Najbolja jedinka dekodirana: [-0.00159741  0.00021458]



<__main__.BinaryUnit instance at 0x1109990e0>

### F7

In [62]:
lower = -50
upper = 150
prec = 4
num_vars = 2
num_cand = 3
pop = 500
mut = 0.05
fun = F7(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=100000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 100000
Fitness najbolje jedinke: 0.0313945859671
Najbolja jedinka: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]
Najbolja jedinka dekodirana: [  9.77516640e-04   2.38418693e-05]



<__main__.BinaryUnit instance at 0x110a1ccb0>

# 2.zad razlicite dimenzionalnosti za F6 i F7

### 1

In [63]:
lower = -50
upper = 150
prec = 4
num_vars = 1
num_cand = 3
pop = 500
mut = 0.05
fun = F6(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=100000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 100000
Fitness najbolje jedinke: 5.68718960814e-10
Najbolja jedinka: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Najbolja jedinka dekodirana: [  2.38418693e-05]



<__main__.BinaryUnit instance at 0x10ff7c518>

In [ ]:
lower = -50
upper = 150
prec = 4
num_vars = 1
num_cand = 3
pop = 500
mut = 0.05
fun = F7(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=100000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 100000
Fitness najbolje jedinke: 0.00540836993605
Najbolja jedinka: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Najbolja jedinka dekodirana: [  2.38418693e-05]



<__main__.BinaryUnit instance at 0x1109be128>

### 3

In [17]:
lower = -50
upper = 150
prec = 4
num_vars = 3
num_cand = 3
pop = 500
mut = 0.05
fun = F6(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=20000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 20000
Fitness najbolje jedinke: 0.0189810469747
Najbolja jedinka: [0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 1 1 0 1 0 1 1
 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1 1]
Najbolja jedinka dekodirana: [ 4.01251507 -2.4058115   4.19037542]



<__main__.BinaryUnit instance at 0x10a33cc20>

In [19]:
lower = -50
upper = 150
prec = 4
num_vars = 3
num_cand = 3
pop = 500
mut = 0.05
fun = F7(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=20000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 20000
Fitness najbolje jedinke: 0.65239661932
Najbolja jedinka: [0 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 1 1 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 0 0]
Najbolja jedinka dekodirana: [-0.35007017  0.03902914 -0.12929446]



<__main__.BinaryUnit instance at 0x10a393d88>

### 6

In [20]:
lower = -50
upper = 150
prec = 4
num_vars = 6
num_cand = 3
pop = 500
mut = 0.05
fun = F6(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=20000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 20000
Fitness najbolje jedinke: 0.193527713418
Najbolja jedinka: [0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0
 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0
 1 1 0 1 1 0 0 1 0 0 0 0 1 0 1]
Najbolja jedinka dekodirana: [ -9.75797642   6.76291311   7.90665527 -14.03492166   1.11162716
  15.14943368]



<__main__.BinaryUnit instance at 0x10adbca70>

In [23]:
lower = -50
upper = 150
prec = 4
num_vars = 6
num_cand = 3
pop = 500
mut = 0.05
fun = F7(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=20000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 20000
Fitness najbolje jedinke: 3.42043328285
Najbolja jedinka: [0 1 0 0 0 1 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1
 0 1 1 0 0 0 1 0 0 0 1 1 1 1 0 0 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1
 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 1
 0 1 0 1 1 0 0 0 1 0 1 1 0 1 1]
Najbolja jedinka dekodirana: [ 5.98280715  0.7915739   5.92739865  3.80928698 -6.63163978 -2.04207995]



<__main__.BinaryUnit instance at 0x10a3f0518>

### 10

In [24]:
lower = -50
upper = 150
prec = 4
num_vars = 10
num_cand = 3
pop = 500
mut = 0.05
fun = F6(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=20000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 20000
Fitness najbolje jedinke: 0.359545141459
Najbolja jedinka: [0 1 0 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0
 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0
 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0 0
 0 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0
 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0]
Najbolja jedinka dekodirana: [ 17.1237312  -33.16611441   3.18396243  24.19007978 -11.36920994
   6.60269575  12.33027569 -14.1726323    2.17402085  -0.10106568]



<__main__.BinaryUnit instance at 0x10ac77488>

In [25]:
lower = -50
upper = 150
prec = 4
num_vars = 10
num_cand = 3
pop = 500
mut = 0.05
fun = F7(num_vars)
ga = GeneticAlg(500,fun,upper,lower,num_vars,prec,num_cand,max_evals=20000,mutation_chance=mut)
ga.run()

######
Broj evaluacija: 20000
Fitness najbolje jedinke: 6.76087808609
Najbolja jedinka: [0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 1 1
 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0
 1 0 0 0 0 1 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0
 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 0 0 0 1 1
 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0]
Najbolja jedinka dekodirana: [ -7.71549116 -21.54673173   3.64086802 -15.57396201  26.65675958
  12.51214147 -10.37662524   1.93998906   6.20606242  12.61389857]



<__main__.BinaryUnit instance at 0x10a37a950>